# Множественная проверка гипотез

In [ ]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

## Foodmart product sales 

In [ ]:
sales = pd.read_csv('foodmart.sales.tsv', sep = '\t', header = 0, parse_dates = [2])

In [ ]:
sales.head()

In [ ]:
products = pd.read_csv('foodmart.products.tsv', sep = '\t', header = 0)

In [ ]:
products.head()

In [ ]:
sales = sales.merge(products[['product_id', 'product_name']], 
                    on = ['product_id'], how = 'inner')

In [ ]:
sales.head()

## Корреляция Пирсона

In [ ]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                     columns=['product_name'], fill_value = 0, aggfunc= lambda x: x)

In [ ]:
sparse_sales.head()

In [ ]:
%%time 
corr_data = []

for i, lhs_column in enumerate(sparse_sales.columns):
    for j, rhs_column in enumerate(sparse_sales.columns):
        if i >= j:
            continue
        
        corr, p = pearsonr(sparse_sales[lhs_column], sparse_sales[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])        

In [ ]:
sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns = ['product_A', 'product_B', 'corr', 'p']

In [ ]:
sales_correlation.head()

Сколько гипотез об отсутствии корреляции отвергается без поправки на множественную проверку?

In [ ]:
(sales_correlation.p < 0.05).value_counts()

## Поправка на множественную проверку

### Метод Холма

In [ ]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [ ]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [ ]:
sales_correlation.head()

In [ ]:
sales_correlation.reject.value_counts()

In [ ]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

### Метод Бенджамини-Хохберга

In [ ]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [ ]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [ ]:
sales_correlation.head()

In [ ]:
sales_correlation.reject.value_counts()

In [ ]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)